In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
mypal = plt.rcParams['axes.prop_cycle'].by_key()['color'] # Grab the color pal
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

In [2]:
exposure = pd.ExcelFile('TBA 2021 R2_Data.xlsx').parse(0,skiprows=6)
exposure

,Age,Exposure ($)
0,30-34,60123311
1,35-39,51000321
2,40-44,98121031
3,45-49,49999832
4,50-54,171133222
5,55-59,179890999


In [3]:
claim_amount = pd.ExcelFile('TBA 2021 R2_Data.xlsx').parse(1,skiprows=5,index_col='Record #').dropna(how='all',axis=1)
claim_amount

,Age,Claim Amount ($)
Record #,,
1,33,8272
2,54,6537
3,54,4563
4,58,4608
5,53,9718
...,...,...
293,45,3346
294,55,5758
295,51,1896


In [4]:
assured = pd.ExcelFile('TBA 2021 R2_Data.xlsx').parse(2,skiprows=6).drop([4,5])
assured['Assured'] = assured['% of Total Sum Assured']*220000000
assured

,Age Exact,% of Total Sum Assured,Assured
0,30-34,0.10,22000000.0
1,35-39,0.32,70400000.0
2,40-44,0.26,57200000.0
3,45-49,0.32,70400000.0


In [5]:
qx = pd.ExcelFile('TBA 2021 R2_Data.xlsx').parse(3,skiprows=8).dropna(how='all',axis=1)
qx

,Age Exact,qx
0,16,0.000171
1,17,0.000175
2,18,0.000180
3,19,0.000186
4,20,0.000192
...,...,...
100,116,0.600518
101,117,0.610721
102,118,0.619979
103,119,0.628289


In [6]:
#Correct data
claim_amount.loc[61,'Claim Amount ($)'] = 6185
claim_amount.loc[44,'Claim Amount ($)'] = 1110

In [7]:
# claim_amount.to_excel('answer1.xlsx',sheet_name='New Claims')

In [8]:
def convert_age_band(age):
    if age < 30:
        return '30-'
    elif age <= 34:
        return '30-34'
    elif age <= 39:
        return '35-39'
    elif age <= 44:
        return '40-44'
    elif age <= 49:
        return '45-49'
    elif age <= 54:
        return '50-54'
    elif age <= 59:
        return '55-59'
    else:
        return '60+'

In [9]:
temp = claim_amount.copy()
temp['Age Band'] = temp['Age'].apply(convert_age_band)
actual_claim = temp.groupby('Age Band')['Claim Amount ($)'].sum().reset_index()
actual_claim

,Age Band,Claim Amount ($)
0,30-,5339
1,30-34,21028
2,35-39,36842
3,40-44,141345
4,45-49,123230
5,50-54,589005
6,55-59,509707
7,60+,10980


In [10]:
mortality_prob = qx.copy()
mortality_prob['Age Band'] = mortality_prob['Age Exact'].apply(convert_age_band)
mortality_prob

,Age Exact,qx,Age Band
0,16,0.000171,30-
1,17,0.000175,30-
2,18,0.000180,30-
3,19,0.000186,30-
4,20,0.000192,30-
...,...,...,...
100,116,0.600518,60+
101,117,0.610721,60+
102,118,0.619979,60+
103,119,0.628289,60+


In [11]:
temp = pd.merge(exposure,mortality_prob,left_on='Age',right_on='Age Band').drop('Age',axis=1)
temp['Claim Amount ($)'] = (temp['Exposure ($)']*temp['qx']).round(0)
expected_claim = temp.groupby('Age Band')['Claim Amount ($)'].mean().reset_index()

In [12]:
actual_claim.rename(columns={'Claim Amount ($)':'Actual Claim Amount ($)'},inplace=True)

In [13]:
expected_claim.rename(columns={'Claim Amount ($)':'Expected Claim Amount ($)'},inplace=True)

In [14]:
ae_ratio = pd.merge(actual_claim,expected_claim,on='Age Band')
ae_ratio.loc[6,'Age Band'] = 'Total'
ae_ratio.loc[6,'Actual Claim Amount ($)'] = ae_ratio['Actual Claim Amount ($)'].sum()
ae_ratio.loc[6,'Expected Claim Amount ($)'] = ae_ratio['Expected Claim Amount ($)'].sum()
ae_ratio['A/E ratio'] = (ae_ratio['Actual Claim Amount ($)']/ae_ratio['Expected Claim Amount ($)']).round(3)
ae_ratio

,Age Band,Actual Claim Amount ($),Expected Claim Amount ($),A/E ratio
0,30-34,21028.0,26502.4,0.793
1,35-39,36842.0,36577.2,1.007
2,40-44,141345.0,115566.8,1.223
3,45-49,123230.0,93960.0,1.312
4,50-54,589005.0,491220.8,1.199
5,55-59,509707.0,742734.2,0.686
6,Total,1421157.0,1506561.4,0.943


In [15]:
# ae_ratio.to_excel('answer2.xlsx',sheet_name='AE Ratio',index=False)

In [16]:
def mortality_adjust(table,ae_ratio_dict):
    new_table = table.copy()
    for i in range(len(ae_ratio_dict)):
        new_table.loc[new_table['Age Band']==list(ae_ratio_dict.keys())[i],'qx'] = new_table.loc[new_table['Age Band']==list(ae_ratio_dict.keys())[i],'qx']\
            * list(ae_ratio_dict.values())[i]
    
    return new_table

In [17]:
qx['Age Band'] = qx['Age Exact'].apply(convert_age_band)
ae_ratio_dict = dict(zip(ae_ratio['Age Band'].drop(6).to_list(),ae_ratio['A/E ratio'].drop(6).to_list()))

new_qx = mortality_adjust(qx,ae_ratio_dict)
new_qx

,Age Exact,qx,Age Band
0,16,0.000171,30-
1,17,0.000175,30-
2,18,0.000180,30-
3,19,0.000186,30-
4,20,0.000192,30-
...,...,...,...
100,116,0.600518,60+
101,117,0.610721,60+
102,118,0.619979,60+
103,119,0.628289,60+


In [18]:
mortality = new_qx.loc[14:34].drop('Age Band',axis=1)
mortality

,Age Exact,qx
14,30,0.000289
15,31,0.000316
16,32,0.000346
17,33,0.000380
18,34,0.000418
19,35,0.000586
20,36,0.000646
21,37,0.000715
22,38,0.000790
23,39,0.000873


In [19]:
qx_list = mortality['qx'].to_list()
age_list = mortality['Age Exact'].to_list()
def get_qx(age,qx_list,age_list):
    return qx_list[age_list.index(age):]

In [20]:
def assurance(rate,begin_term,qx_list):
    last = len(qx_list)-1

    if begin_term == last:
        return 1/(1+rate)
    else:
        return (1/(1+rate))*(qx_list[begin_term]+(1-qx_list[begin_term])*assurance(rate,begin_term+1,qx_list))

In [21]:
mortality['Assurance Factor'] = [assurance(0.03,0,get_qx(i,qx_list,age_list)) for i in age_list]

In [22]:
def annuity(rate,begin_term,qx_list):
    last = len(qx_list)-1

    if begin_term == last:
        return 1
    else:
        return 1+((1/(1+rate))*(1-qx_list[begin_term])*annuity(rate,begin_term+1,qx_list))

In [23]:
# [annuity(0.04,0,get_qx(i,[0.002809,0.003152,0.003539,0.003976],[51,52,53,54])) for i in [51,52,53,54]]
mortality['Annuity Factor'] = [annuity(0.03,0,get_qx(i,qx_list,age_list)) for i in age_list]

In [24]:
mortality

,Age Exact,qx,Assurance Factor,Annuity Factor
14,30,0.000289,0.540668,15.770382
15,31,0.000316,0.556761,15.217886
16,32,0.000346,0.573329,14.649046
17,33,0.000380,0.590387,14.063380
18,34,0.000418,0.607950,13.460394
19,35,0.000586,0.626032,12.839572
20,36,0.000646,0.644605,12.201910
21,37,0.000715,0.663725,11.545432
22,38,0.000790,0.683411,10.869566
23,39,0.000873,0.703679,10.173695


In [25]:
# mortality.to_excel('answer3.xlsx',sheet_name='Adj qx - Assurance - Annuity',index=False)

In [26]:
temp = pd.DataFrame()
temp['Age'] = mortality['Age Exact']
temp['PV Of Claim Outgo'] = (mortality['Assurance Factor']*10000).round(3)
temp['PV of $1'] = (mortality['Annuity Factor']*1000).round(3)
temp['Age Band'] = temp['Age'].apply(convert_age_band)
temp = temp.groupby('Age Band').mean().reset_index().drop('Age',axis=1).drop(4)
temp

,Age Band,PV Of Claim Outgo,PV of $1
0,30-34,5738.1892,14632.2176
1,35-39,6642.9026,11526.0350
2,40-44,7687.3068,7940.2470
3,45-49,8897.2218,3786.2056


In [27]:
premium_2022 = expected_claim.copy().drop([4,5])
premium_2022['Gross Premium'] = (premium_2022['Expected Claim Amount ($)']/(1-0.3)).round(3)
premium_2022['Net Premium'] = premium_2022['Gross Premium']-temp['PV Of Claim Outgo']-temp['PV of $1']
premium_2022.drop('Expected Claim Amount ($)',axis=1,inplace=True)
premium_2022.loc[4,'Age Band'] = 'Overall Average'
premium_2022.loc[4,1:] = premium_2022.mean(axis=0)
premium_2022

,Age Band,Gross Premium,Net Premium
0,30-34,37860.5710,17490.1642
1,35-39,52253.1430,34084.2054
2,40-44,165095.4290,149467.8752
3,45-49,134228.5710,121545.1436
4,Overall Average,97359.4285,80646.8471


In [28]:
# premium_2022.to_excel('answer4.xlsx',sheet_name='Premium 2022',index=False)